## **Deep Learning**

In [ ]:
!pip install hickle
import hickle as hkl
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import layers,Input,Sequential
from keras.layers import Dense,Flatten,Reshape,Conv2DTranspose,Conv2D,MaxPooling2D,Reshape,Resizing
from tensorflow.keras import datasets, layers, models, losses, Model
from random import randint
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
load_hkl = hkl.load('/gdrive/My Drive/BitirmeTezi/image_dataset.hkl')
X_train=load_hkl['xtrain'] / 255.
X_test=load_hkl['xtest'] / 255.
y_train=load_hkl['ytrain']
y_test=load_hkl['ytest']

In [ ]:
X_train.shape

(427, 200, 200, 3)

In [ ]:
X_test.shape

(183, 200, 200, 3)

## **Burada Rescaling Layer autoencoder da olmaz çünkü xtrain verilseydi, normalize edilerek kullanılırdı ama çıkışta normalize edilmemiş xtrain ile karşılaştırılırdı !!!!!**

In [ ]:
class CNN_Autoencoder(Model):
    def __init__(self):
        super(CNN_Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=X_train.shape[1:]),
            layers.Conv2D(8, 11, strides=5, padding='same', activation='relu'),
            layers.Conv2D(16, 5, strides=2, padding='same', activation='relu'),
            layers.Conv2D(32, 3, strides=1, padding='same', activation='relu')])

        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(32, 3, strides=1, padding='same', activation='relu'),
            layers.Conv2DTranspose(16, 3, strides=2, padding='same', activation='relu'),
            layers.Conv2DTranspose(3, 3, strides=5, padding='same', activation='sigmoid')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
cnn_autoencoder=CNN_Autoencoder()
cnn_autoencoder.compile(optimizer='adam',  loss='mse')

In [ ]:
cnn_autoencoder.encoder.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_102 (Conv2D)         (None, 40, 40, 8)         2912      
                                                                 
 conv2d_103 (Conv2D)         (None, 20, 20, 16)        3216      
                                                                 
 conv2d_104 (Conv2D)         (None, 20, 20, 32)        4640      
                                                                 
Total params: 10768 (42.06 KB)
Trainable params: 10768 (42.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history=cnn_autoencoder.fit(X_train,X_train,epochs=10,batch_size=20,shuffle=True,validation_data=(X_test,X_test))

Epoch 1/10
22/22 [==============================] - 6s 216ms/step - loss: 0.0537 - val_loss: 0.0495
Epoch 2/10
22/22 [==============================] - 5s 236ms/step - loss: 0.0448 - val_loss: 0.0410
Epoch 3/10
22/22 [==============================] - 5s 245ms/step - loss: 0.0393 - val_loss: 0.0377
Epoch 4/10
22/22 [==============================] - 4s 193ms/step - loss: 0.0368 - val_loss: 0.0357
Epoch 5/10
22/22 [==============================] - 5s 238ms/step - loss: 0.0350 - val_loss: 0.0342
Epoch 6/10
22/22 [==============================] - 5s 235ms/step - loss: 0.0336 - val_loss: 0.0329
Epoch 7/10
22/22 [==============================] - 4s 192ms/step - loss: 0.0325 - val_loss: 0.0320
Epoch 8/10
22/22 [==============================] - 5s 240ms/step - loss: 0.0316 - val_loss: 0.0311
Epoch 9/10
22/22 [==============================] - 6s 258ms/step - loss: 0.0308 - val_loss: 0.0303
Epoch 10/10
22/22 [==============================] - 4s 190ms/step - loss: 0.0301 - val_loss: 0.0297

In [ ]:
cnn_autoencoder.decoder.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose (Conv2DTr  (None, 20, 20, 32)        9248      
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 40, 40, 16)        4624      
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 200, 200, 3)       435       
 Transpose)                                                      
                                                                 
Total params: 14307 (55.89 KB)
Trainable params: 14307 (55.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
xm_train=cnn_autoencoder.encoder(X_train)
xm_test=cnn_autoencoder.encoder(X_test)


In [ ]:
xm_train=tf.reshape(xm_train,[427,-1])

In [ ]:
xm_test=tf.reshape(xm_test,[183,-1])

In [ ]:
cnn_autoencoder.save('/gdrive/My Drive/BitirmeTezi/image_dataset_autoencoder.keras')

## **Machine Learning**

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

xm_train=scaler.fit_transform(xm_train)
xm_test=scaler.transform(xm_test)

In [ ]:
from sklearn.svm import SVC

svc=SVC(kernel="rbf")
svc.fit(xm_train, y_train)

SVC()

In [ ]:
y_pred=svc.predict(xm_test)

from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test, y_pred)
print(cm)


[[40  3  8]
 [ 3 56 11]
 [ 5  8 49]]


In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb=GaussianNB()
gnb.fit(xm_train, y_train)

y_pred=gnb.predict(xm_test)
cm=confusion_matrix(y_test, y_pred)
print(cm)

[[11  7 33]
 [ 8 26 36]
 [ 5  9 48]]


In [ ]:
from sklearn.naive_bayes import ComplementNB

cnb=ComplementNB()
cnb.fit(xm_train, y_train)

y_pred=cnb.predict(xm_test)
cm=confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=5,metric='minkowski')

knn.fit(xm_train, y_train)
y_pred=knn.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)

[[38  0 13]
 [16 33 21]
 [13  4 45]]


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc=DecisionTreeClassifier(criterion="entropy")

dtc.fit(xm_train, y_train)
y_pred=dtc.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)

[[36  7  8]
 [10 48 12]
 [ 9  9 44]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(n_estimators=30, criterion="entropy", random_state=0)

rfc.fit(xm_train, y_train)
y_pred=rfc.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)

[[43  1  7]
 [ 7 57  6]
 [ 5  5 52]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

xm_train=scaler.fit_transform(xm_train)
xm_test=scaler.transform(xm_test)


from sklearn.naive_bayes import MultinomialNB

mnb=MultinomialNB()
mnb.fit(xm_train, y_train)

y_pred=mnb.predict(xm_test)

cm=confusion_matrix(y_test, y_pred)
print(cm)


[[19 20 12]
 [19 38 13]
 [22 17 23]]


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))